In [38]:
import openai
import langchain
import pinecone 
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI


In [64]:
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
import os

In [41]:
def read_doc(directory):
    file_loader=CSVLoader(directory)
    documents=file_loader.load()
    return documents

In [42]:
doc = read_doc("gym_recommendation.csv")

In [43]:
doc

[Document(metadata={'source': 'gym_recommendation.csv', 'row': 0}, page_content="ID: 1\nSex: Male\nAge: 18\nHeight: 1.68\nWeight: 47.5\nHypertension: No\nDiabetes: No\nBMI: 16.83\nLevel: Underweight\nFitness Goal: Weight Gain\nFitness Type: Muscular Fitness\nExercises: Squats, deadlifts, bench presses, and overhead presses\nEquipment: Dumbbells and barbells\nDiet: Vegetables: (Carrots, Sweet Potato, and Lettuce); Protein Intake: (Red meats, poultry, fish, eggs, dairy products, legumes, and nuts); Juice: (Fruit juice, watermelon juice, carrot juice, apple juice and mango juice)\nRecommendation: Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. It is important to approach weight gain in a healthy and balanced way, focusing on both exercise and nutrition. Keep in mind that weight gain should be gradual and focused on building lean muscle rather than increasing fat. Additionally, it's always a good idea to consult with a health care professiona

In [44]:
def chunk_data(docs,chunk_size=800,chunk_overlap=0):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [45]:
documents=chunk_data(docs=doc)
len(documents)

14589

In [46]:
documents

[Document(metadata={'source': 'gym_recommendation.csv', 'row': 0}, page_content="ID: 1\nSex: Male\nAge: 18\nHeight: 1.68\nWeight: 47.5\nHypertension: No\nDiabetes: No\nBMI: 16.83\nLevel: Underweight\nFitness Goal: Weight Gain\nFitness Type: Muscular Fitness\nExercises: Squats, deadlifts, bench presses, and overhead presses\nEquipment: Dumbbells and barbells\nDiet: Vegetables: (Carrots, Sweet Potato, and Lettuce); Protein Intake: (Red meats, poultry, fish, eggs, dairy products, legumes, and nuts); Juice: (Fruit juice, watermelon juice, carrot juice, apple juice and mango juice)\nRecommendation: Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. It is important to approach weight gain in a healthy and balanced way, focusing on both exercise and nutrition. Keep in mind that weight gain should be gradual and focused on building lean muscle rather than increasing fat. Additionally, it's always a good idea to consult with a health care professiona

In [47]:
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"

In [50]:

embeddings=HuggingFaceEmbeddings(model_name=embedding_model_name)
embeddings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [54]:
vector_dimension = embeddings.embed_query("Test query")
print(len(vector_dimension))

384


In [55]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
#index = pc.Index("gymrecommendation")

In [56]:
index_name = "gymrecommendation-huggingface"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [57]:
from langchain_pinecone import PineconeVectorStore


In [58]:
vectorestore = PineconeVectorStore.from_documents(documents,embeddings,index_name=index_name)

In [69]:
def retrieve_query(query,k=2):
    matching_results =vectorestore.similarity_search(query,k=k)
    return matching_results

In [62]:
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq

In [65]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",   # Specify the model name
    temperature=1,                     # Set the desired temperature
    max_tokens=7999,                   # Define the maximum number of tokens
    timeout=10,                        # Set a timeout in seconds
    max_retries=2                      # Number of retries in case of errors
)

chain = load_qa_chain(llm,chain_type="stuff")

In [71]:
def retrieve_answer(query):
    ans_search = retrieve_query(query)
    print(ans_search)
    response = chain.run(input_documents=ans_search,question=query)
    return response

In [72]:
our_query = "I am a 24 year old male and I would like to gain muscle and reduce fat can you give me my excercise plan"
answer = retrieve_answer(our_query)
print(answer)

[Document(id='00d6f59b-7aa7-4a25-a60f-2d851b6d6017', metadata={'row': 228.0, 'source': 'gym_recommendation.csv'}, page_content="ID: 229\nSex: Male\nAge: 25\nHeight: 1.62\nWeight: 50.0\nHypertension: No\nDiabetes: No\nBMI: 19.05\nLevel: Normal\nFitness Goal: Weight Gain\nFitness Type: Muscular Fitness\nExercises: Squats, deadlifts, bench presses, and overhead presses\nEquipment: Dumbbells and barbells\nDiet: Vegetables: (Carrots, Sweet Potato, Lettuce); Protein Intake: (Red meats, poultry, fish, eggs, dairy products, legumes, and nuts); Juice: (Fruit juice, watermelon juice, carrot juice, apple juice and mango juice)\nRecommendation: Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. It is important to approach weight gain in a healthy and balanced way, focusing on both exercise and nutrition. Keep in mind that weight gain should be gradual and focused on building lean muscle rather than increasing fat. Additionally, it's always a good idea t

In [73]:
our_query = "im 25 year male, as I want to gain muscle and weight in 40 days give me diet"
answer = retrieve_answer(our_query)
print(answer)


[Document(id='bdba3e7e-948d-46c7-a7b1-0fc998f9184d', metadata={'row': 4097.0, 'source': 'gym_recommendation.csv'}, page_content="ID: 4098\nSex: Male\nAge: 40\nHeight: 1.84\nWeight: 46.5\nHypertension: No\nDiabetes: No\nBMI: 13.73\nLevel: Underweight\nFitness Goal: Weight Gain\nFitness Type: Muscular Fitness\nExercises: Squats, deadlifts, bench presses, and overhead presses\nEquipment: Dumbbells and barbells\nDiet: Vegetables: (Carrots, Sweet Potato, and Lettuce); Protein Intake: (Red meats, poultry, fish, eggs, dairy products, legumes, and nuts); Juice: (Fruit juice, watermelon juice, carrot juice, apple juice and mango juice)\nRecommendation: Follow a regular exercise schedule. Adhere to the exercise and diet plan to get better results. It is important to approach weight gain in a healthy and balanced way, focusing on both exercise and nutrition. Keep in mind that weight gain should be gradual and focused on building lean muscle rather than increasing fat. Additionally, it's always a 